# 0. Short description 

In the first part of the lab you are going to implement a standard document processing pipeline and then build a simple search engine based on it:
- starting from crawling documents, 
- then building an inverted index,
- and answering queries using this index.

Second part is devoted to spellchecking. You will implement some functions related to spellchecking.
- K-gram index
- weighted editorial distance
- Norvig spellchecker


# 1. [60] Building inverted index and answering queries

## 1.1. [10] Preprocessing

First, we need a unified approach to documents preprocessing. Implement a class responsible for that. Complete the code for given functions (most of them are just one-liners) and make sure you pass the tests. Make use of `nltk` library or any other you know.

In [1]:
import nltk
nltk.download('punkt')

class Preprocessor:
    
    def __init__(self):
        self.stop_words = {'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
                      'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with'}
        self.ps = nltk.stem.PorterStemmer()

    
    def tokenize(self, text):
        #TODO word tokenize text using nltk lib
        return nltk.word_tokenize(text)

    
    def stem(self, word, stemmer):
        #TODO stem word using provided stemmer
        return stemmer.stem(word)

    
    def is_apt_word(self, word):
        #TODO check if word is appropriate - not a stop word and isalpha, 
        # i.e consists of letters, not punctuation, numbers, dates
        return word not in self.stop_words and str.isalpha(word)

    
    def preprocess(self, text):
        #TODO combine all previous methods together: tokenize lowercased text 
        # and stem it, ignoring not appropriate words
        result  = [self.stem(token,self.ps) for token in  self.tokenize(text.lower()) if self.is_apt_word(token)]
        return result

[nltk_data] Downloading package punkt to /home/ravio_o/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 1.1.1. Tests

In [2]:
prep = Preprocessor()
text = 'To be, or not to be, that is the question'

assert prep.tokenize(text) == ['To', 'be', ',', 'or', 'not', 'to', 'be', ',', 'that', 'is', 'the', 'question']
assert prep.stem('retrieval', prep.ps) == 'retriev'
assert prep.is_apt_word('qwerty123') is False
assert prep.preprocess(text) == ['or', 'not', 'question']

## 1.2. [35] Crawling and Indexing

### 1.2.1. [10] Base classes

Here are some base classes you will need for writing an indexer. The code from the first lab's solution is given, still you need to change some of it, namely, the `parse` function (it is also possible to use your own implementation from the first homework, but make sure that it is correct). The reason is it always makes complete parsing, which we want to avoid when we only need links, for example, or a specific portion of text.

In [3]:
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse
import os

class Document:

    def __init__(self, url):
        self.url = url
        self.download()

    def download(self):
        try:
            response = requests.get(self.url)
            if response.status_code == 200:
                self.content = response.content
                return True
            else:
                return False
        except:
            return False

    def persist(self, path):
        os.makedirs(path, exist_ok= True)
        with open(os.path.join(path, quote(self.url).replace('/', '_')), 'wb') as f:
            f.write(self.content)


class HtmlDocument(Document):

    def normalize(self, href):
        if href is not None and href[:4] != 'http':
            href = urllib.parse.urljoin(self.url, href)
        return href

    def parse_anchors(self, model):
        anchors = []
        a = model.find_all('a')
        for anchor in a:
            href = self.normalize(anchor.get('href'))
            text = anchor.text
            if href == None:
                continue
            anchors.append((text, href))
        self.anchors = anchors
        return

    def parse_images(self, model):
        images = []
        i = model.find_all('img')
        for img in i:
            href = self.normalize(img.get('src'))
            images.append(href)
        self.images = images
        return

    def parse_text(self, model):
        texts = model.findAll(text=True)
        visible_texts = filter(self.tag_visible, texts)
        self.text = u" ".join(t.strip() for t in visible_texts)
        return

    def tag_visible(self, element):
        if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
            return False
        if isinstance(element, Comment):
            return False
        return True

    def parse(self, parse_links = False, parse_img = False, parse_text = False):
        #TODO change this method
        model = BeautifulSoup(self.content )
        if parse_links: self.parse_anchors(model)
        elif parse_img: self.parse_images(model)
        elif parse_text: self.parse_text(model)

### 1.2.2. [20] Main class

The main indexer logic is here. We organize it as a crawler generator that adds certain visited pages to inverted index and saves them on disk. 

- `crawl_generator_for_index` method crawles the given website doing BFS, starting from `source` within given `depth`. Considers only inner pages (of a form https://www.reuters.com/...) for visiting. To speed up, doesn't consider for visiting pages with content type other than html: '.pdf', '.mp3', '.avi', '.mp4', '.txt'. If encounters an article page (of a form https://www.reuters.com/article/...), saves its content in a file in `collection_path` folder and populates the inverted index calling `index_doc` method. When done, saves on disk three resulting dictionaries:
    - `doc_urls`, `doc_id:url`
    - `index`, `term:[collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]`
    - `doc_lengths`, `doc_id:doc_length` 

    `limit` parameter is given for testing - if not `None`, break the loop when number of saved articles exceeds the `limit` and return without writing dictionaries to disk.
    
    
- `index_doc` method parses and preprocesses the content of a `doc` and adds it to the inverted index. Also keeps track of document lengths in a `doc_lengths` dictionary.

Your crawler have to print visited urls as it runs.

In [4]:
from collections import Counter
from queue import Queue
import pickle
import os
from urllib.parse import urlparse

class Indexer:

    def __init__(self):      
        # dictionaries to populate
        self.doc_urls = {}        
        self.index = {}
        self.doc_lengths = {}
        self.doc_id = 1
        # preprocessor
        self.prep = Preprocessor()

    def if_html(self, link):
        filter = ['.pdf', '.mp3', '.avi', '.mp4', '.txt']
        url_parsed = urlparse(link)
        for f in filter:
            if f in url_parsed:
                return False
        return True

    def index_article(self, article):
        article.parse(parse_text=True)
        text = self.prep.preprocess(article.text)
        self.doc_urls[self.doc_id] = article.url
        self.doc_lengths[self.doc_id] = len(text)

        for term in text:
            if term not in self.index:
                list = [1]
                list.append((self.doc_id,1))
                self.index[term] = list
            else:
                coll_freq, articles_list = self.index[term][0], self.index[term][1:]
                coll_freq +=1

                # to manipulate with list of tuples convert to dict with key: doc_id
                article_dict = dict(articles_list)
                # Increase frequency of the term in specified article(doc_id)
                if self.doc_id in article_dict:
                    article_dict[self.doc_id] += 1
                else:
                    article_dict[self.doc_id] = 1

                list = [coll_freq]
                list.extend([(k,v) for k,v in article_dict.items()])
                self.index[term] = list
        self.doc_id += 1

    def remove_tail(self, link):
        if (link[-1] == '/'):
            return link[:-1]
        else:
            return link

    def limit(self, limit):
        if limit:
            return len(self.doc_urls) < limit
        else:
            return True

    def crawl_generator_for_index(self, source, depth, collection_path="collection", limit=None):        
        #TODO generate url-s for visiting

        queue = []
        visited = set()
        depth_level_queue = []

        # Initial links
        src = HtmlDocument(source)
        visited.add(self.remove_tail(source))
        src.parse(parse_links=True)

        if "https://www.reuters.com/article/" in source:
            src.persist(collection_path)
        self.index_article(src)
        yield src
        for _, link in src.anchors:
            link = self.remove_tail(link)
            if ("https://www.reuters.com" in link) and (self.if_html(link)) and (link not in visited) and (link not in queue):
                queue.append(link)

        # Go in depth
        for level in range(depth - 1):
            print(level)
            print(len(queue))
            while(queue):
                try:
                    link = queue.pop(0)
                    if link not in visited:
                        src = HtmlDocument(link)
                        visited.add(link)
                        src.parse(parse_links=True)
                        for _, anchor in src.anchors:
                            anchor = self.remove_tail(anchor)
                            if ("https://www.reuters.com" in anchor) and anchor not in depth_level_queue and (self.if_html(anchor)) and (anchor not in visited) and (anchor not in queue):
                                depth_level_queue.append(anchor)
                        if not self.limit(limit):
                            return
                        elif "https://www.reuters.com/article/" in link:
                                src.persist(collection_path)
                        self.index_article(src)
                        yield src
                       
                except Exception as e:
                    print(e)
            print(queue)
            queue.extend(depth_level_queue)
            depth_level_queue=[]

        # save results for later use
        with open('doc_urls.p', 'wb') as fp:
            pickle.dump(self.doc_urls, fp, protocol=pickle.HIGHEST_PROTOCOL)
        with open('inverted_index.p', 'wb') as fp:
            pickle.dump(self.index, fp, protocol=pickle.HIGHEST_PROTOCOL)
        with open('doc_lengths.p', 'wb') as fp:
            pickle.dump(self.doc_lengths, fp, protocol=pickle.HIGHEST_PROTOCOL)

### 1.2.3. Tests

Please make sure your crawler prints out urls with `print(k, c.url)`

In [5]:

indexer = Indexer()
k = 1
for c in indexer.crawl_generator_for_index("https://www.reuters.com/technology", 2, "test_collection", 5):
    print(k, c.url)
    k += 1

assert type(indexer.index) is dict
assert type(indexer.index['reuter']) is list
assert type(indexer.index['reuter'][0]) is int
assert type(indexer.index['reuter'][1]) is tuple

1 https://www.reuters.com/technology
0
70
2 https://www.reuters.com
3 https://www.reuters.com/home
4 https://www.reuters.com/world
5 https://www.reuters.com/world/us-politics


### 1.2.4. Building an index

In [6]:
indexer = Indexer()
k = 1
for c in indexer.crawl_generator_for_index("https://www.reuters.com/", 3, "docs_collection"):
    print(k, c.url)
    k += 1

1 https://www.reuters.com/
0
119
2 https://www.reuters.com/home
3 https://www.reuters.com/world
4 https://www.reuters.com/world/us-politics
5 https://www.reuters.com/world/us
6 https://www.reuters.com/world/uk
7 https://www.reuters.com/world/china
8 https://www.reuters.com/world/india
9 https://www.reuters.com/world/americas
10 https://www.reuters.com/world/asia-pacific
11 https://www.reuters.com/world/europe
12 https://www.reuters.com/world/middle-east-africa
13 https://www.reuters.com/world/the-great-reboot
14 https://www.reuters.com/business
15 https://www.reuters.com/business/sustainable-business
16 https://www.reuters.com/business/legal
17 https://www.reuters.com/business/energy
18 https://www.reuters.com/business/environment
19 https://www.reuters.com/business/finance
20 https://www.reuters.com/business/media-telecom
21 https://www.reuters.com/business/healthcare-pharmaceuticals
22 https://www.reuters.com/business/autos-transportation
23 https://www.reuters.com/business/aerospace

### 1.2.5. [5] Index statistics

Load an index and print the statistics.

In [7]:
# load index, doc_lengths and doc_urls
with open('inverted_index.p', 'rb') as fp:
    index = pickle.load(fp)
with open('doc_lengths.p', 'rb') as fp:
    doc_lengths = pickle.load(fp)
with open('doc_urls.p', 'rb') as fp:
    doc_urls = pickle.load(fp)
    
    
print('Total index length', len(index))
print('\nTop terms by number of documents they apperared in:')
sorted_by_n_docs = sorted(index.items(), key=lambda kv: (len(kv[1]), kv[0]), reverse=True)
print([(sorted_by_n_docs[i][0], len(sorted_by_n_docs[i][1])) for i in range(20)])
print('\nTop terms by overall frequency:')
sorted_by_freq = sorted(index.items(), key=lambda kv: (kv[1][0], kv[0]), reverse=True)
print([(sorted_by_freq[i][0], sorted_by_freq[i][1][0]) for i in range(20)])

Total index length 15406

Top terms by number of documents they apperared in:
[('use', 1014), ('term', 1014), ('privaci', 1014), ('reuter', 1013), ('not', 984), ('see', 983), ('all', 982), ('do', 980), ('here', 979), ('us', 976), ('my', 975), ('person', 974), ('exchang', 973), ('list', 972), ('site', 971), ('sell', 971), ('inform', 971), ('complet', 970), ('delay', 968), ('minut', 967)]

Top terms by overall frequency:
[('reuter', 6879), ('s', 4295), ('said', 3173), ('all', 2864), ('market', 2716), ('us', 2331), ('not', 2324), ('delay', 2309), ('minut', 2001), ('advertis', 1969), ('news', 1924), ('busi', 1869), ('after', 1848), ('world', 1734), ('use', 1713), ('follow', 1676), ('new', 1652), ('more', 1550), ('inform', 1545), ('thomson', 1543)]


## 1.3. [15] Answering a query (finally)

Now, given that we already have built the inverted index, it's time to utilize it for answering user queries. In this class there are two methods you need to implement:
- `boolean_retrieval`, the simplest form of document retrieval which returns a set of documents such that each one contains all query terms. Returns a set of document ids. Refer to *ch.1* of the book for details;
- `okapi_scoring`, Okapi BM25 ranking function - assigns scores to documents in the collection that are relevant to the user query. Returns a dictionary of scores, `doc_id:score`. Read about it in [Wikipedia](https://en.wikipedia.org/wiki/Okapi_BM25#The_ranking_function) and implement accordingly.

Both methods accept `query` parameter in a form of a dictionary, `term:frequency`

In [8]:
from collections import Counter
import math

class QueryProcessing:
    
    @staticmethod
    def prepare_query(raw_query):
        prep = Preprocessor()
        # pre-process query the same way as documents
        query = prep.preprocess(raw_query)
        # count frequency
        return Counter(query)
    
    @staticmethod
    def boolean_retrieval(query, index):
        docs = None
        for term in query:
            if docs:
                docs = docs.intersection(set([doc[0] for doc in index[term][1:]]))
            else:
                docs = set([doc[0] for doc in index[term][1:]])
        return docs

    
    @staticmethod
    def okapi_scoring(query, doc_lengths, index, k1=1.2, b=0.75):
        #TODO retrieve relevant documents with scores

        docs_scores = dict()
        for doc in doc_lengths.keys():
            docs_scores[doc] = 0

        avgdl = sum(doc_lengths.values()) / len(doc_lengths)
        for term, q_freq in query.items():
            if term in index:
                idf = math.log10(len(doc_lengths) / len(index[term][1]))

                for doc_id, doc_freq in index[term][1:]:
                    docs_scores[doc_id] += idf * doc_freq * (k1 + 1) / (doc_freq + k1 * (1 - b + b * doc_lengths[doc_id]/avgdl))
        return dict([(k,v) for k,v in docs_scores.items() if v > 0])

### 1.3.1. Tests

In [9]:
test_doc_lengths = {1: 20, 2: 15, 3: 10, 4:20, 5:30}
test_index = {'x': [2, (1, 1), (2, 1)], 'y': [2, (1, 1), (3, 1)], 'z': [3, (2, 1), (4,2)]}


test_query1 = QueryProcessing.prepare_query('x z')
test_query2 = QueryProcessing.prepare_query('x y')


assert QueryProcessing.boolean_retrieval(test_query1, test_index) == {2}
assert QueryProcessing.boolean_retrieval(test_query2, test_index) == {1}
okapi_res = QueryProcessing.okapi_scoring(test_query2, test_doc_lengths, test_index)
assert all(k in okapi_res for k in (1, 2, 3))
assert not any(k in okapi_res for k in (4, 5))
assert okapi_res[1] > okapi_res[3] > okapi_res[2]

# 2. [40] Sorri not veri gud in inglish

Have you ever googled someone's name without knowing exactly how it should be written? Were you ever reluctant to look up the correct spelling of a query you typed? Or just unable to type properly because of being in a rush? Modern search engines usually do a pretty good job in deciphering defective user input. In order to be able to do that, a good spell-checking mechanism should be incorporated into a search procedure. Today we will take one step further towards building a good search engine and work on tolerant retrieval with respect to user queries. We will consider two cases:

1. Users know that they don't know the correct spelling OR they want to get the results that follow some known pattern, so the use so called wildcards - queries like `retr*val`;
2. Users don't know the correct spelling OR they don't care OR they are in a rush OR they expect that mistakes will be corrected OR /your option/... so they make mistakes and we need to handle them using:

    2.1. Trigrams with Jaccard coefficient;
    
    2.2. Simple spellchecker by Peter Norvig with QWERTY weights;

## 2.1. [25] Handling wildcards

We will handle wildcard queries using k-grams. K-gram is a list of consecutive k chars in a string - i.e., for the word *'star'*, it will be '*\$st*', '*sta*', '*tar*', and '*ar$*', if we take k=3. Take a look at [book](https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf) *chapter 3.2.2* to understand how k-grams can help efficiently match a wildcard against dictionary words. Here we will only consider wildcards with star symbols (may be multiple).

Notice that for building k-grams index, **we will need a vocabulary of original word forms** to compare words in user input to the vocabulary of "correct" words (think why inverted index which we built for stemmed words doesn't work here).   

You need to implement the following:

- `build_inverted_index_orig_forms` - creates inverted index of original world forms from `facts` list, which is already given to you.  
    Output format: `term:[collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]`
    

- `build_k_gram_index` - creates k-gram index which maps every k-gram encountered in facts collection to a list of words containing this k-gram. Use the abovementioned inverted index of original words to construct this index.  
    Output format: `'k_gram': ['word1_with_k_gram', 'word2_with_k_gram', ...]`
    
    
- `generate_wildcard_options` - produce a list of vocabulary words matching given wildcard by intersecting postings of k-grams present in the wildcard (refer to *ch 3.2.2*). 

- `search_wildcard` - return list of facts that contain the words matching a wildcard query.


We will use the dataset with curious facts for testing.

### 2.1.1. Downloading the dataset

In [16]:
import urllib.request
data_url = "https://raw.githubusercontent.com/IUCVLab/information-retrieval/main/datasets/facts.txt"
local_filename, headers = urllib.request.urlretrieve(data_url)

facts = []
with open(local_filename) as fp:
    for cnt, line in enumerate(fp):
        facts.append(line.strip('\n'))
        
print(*facts[-5:], sep='\n')

151. Women have twice as many pain receptors on their body than men. But a much higher pain tolerance.
152. There are more stars in space than there are grains of sand on every beach in the world.
153. For every human on Earth there are 1.6 million ants.
154. The total weight of all those ants, however, is about the same as all the humans.
155. On Jupiter and Saturn it rains diamonds.


### 2.1.2. [25] Implementation of search

In [17]:
import re

def tokenize(text):
    #TODO word tokenize text using nltk lib
    return nltk.word_tokenize(text)

def build_inverted_index_orig_forms(documents):
    #TODO build an inverted index of original word forms 
    # (without stemming, just word tokenized and lowercased)   
    inverted_index = {}
    for id, line in enumerate(documents):
        tokens = tokenize(line)
        tokens = [token.lower() for token in tokens]
        for term in tokens:
            if term not in inverted_index:
                list = [1]
                list.append((id, 1))
                inverted_index[term] = list
            else:
                coll_freq, docs_list = inverted_index[term][0], inverted_index[term][1:]
                coll_freq += 1

                docs_dict = dict(docs_list)
                if id not in docs_dict.keys():
                    docs_dict[id] = 1
                else:
                    docs_dict[id] += 1
                list = [coll_freq]
                list.extend([(k,v) for k,v in docs_dict.items()])
                inverted_index[term] = list
    return inverted_index


def build_k_gram_index(inverted_index, k):
    #TODO build index of k-grams for dictionary words. 
    # Padd with '$' ($word$) before splitting to k-grams    
    k_gram_index = {}

    for word in inverted_index:
        term = '$' + word + '$'
        k_gram_list = [term[i:i+k] for i in range(len(term)-k+1)]
        for gram in k_gram_list:
            if gram not in k_gram_index:
                k_gram_index[gram] = [word]
            else:
                k_gram_index[gram].append(word)
    return k_gram_index


def generate_wildcard_options(wildcard, k_gram_index, inverted_index):
    #TODO for a given wildcard return all words matching it using k-grams
    # refer to book chapter 3.2.2
    # don't forget to pad wildcard with '$', when appropriate
    word = '$' + wildcard + '$'
    # Identify what k value was
    k = len(list(k_gram_index.keys())[0])
    # Divide for such k_grams
    k_gram_list = [word[i:i+k] for i in range(len(word)-k+1)]

    filtered =  [gram for gram in k_gram_list if gram != '$' and '*' not in gram]
    candidates = set()
    for gram in filtered:
        if gram in k_gram_index:
            if candidates:
                candidates.intersection_update(k_gram_index[gram])
            else:
                candidates.update(k_gram_index[gram])
    # Post filtering
    word = '^' + wildcard.replace('*','.*') + '$'
    return list([candidate for candidate in candidates if re.match(word,candidate)])


def search_wildcard(wildcard, k_gram_index, index, docs):
    #TODO retrive list of documnets (facts) that contain words matching wildcard
    query = generate_wildcard_options(wildcard,k_gram_index,index)
    docs_id = None
    for term in query:
        if docs_id:
            docs_id = docs_id.union(set(doc[0] for doc in index[term][1:]))
        else:
            docs_id = set(doc[0] for doc in index[term][1:])
    if not docs_id:
        return []

    return [fact for doc_id,fact in enumerate(docs) if doc_id in docs_id]


### 2.1.3. Tests

In [18]:
index_orig_forms = build_inverted_index_orig_forms(facts)
k_gram_index = build_k_gram_index(index_orig_forms, 3)

wildcard = "re*ed"

wildcard_options = generate_wildcard_options(wildcard, k_gram_index, index_orig_forms)
print(wildcard_options)
assert(len(wildcard_options) >= 3)
assert("red" not in wildcard_options) 

wildcard_results = search_wildcard(wildcard, k_gram_index, index_orig_forms, facts)
# some pretty printing
for r in wildcard_results:
    # highlight terms for visual evaluation
    for term in wildcard_options:
        r = re.sub(r'(' + term + ')', r'\033[1m\033[91m\1\033[0m', r, flags=re.I)
    print(r)

assert(len(wildcard_results) >=3)

assert "13. James Buchanan, the 15th U.S. president continuously bought slaves with his own money in order to free them." in search_wildcard("pres*dent", k_gram_index, index_orig_forms, facts)
assert "40. 9 out of 10 Americans are deficient in Potassium." in search_wildcard("p*tas*um", k_gram_index, index_orig_forms, facts)
assert "61. A man from Britain changed his name to Tim Pppppppppprice to make it harder for telemarketers to pronounce." in search_wildcard("*price", k_gram_index, index_orig_forms, facts)

['received', 'reduced', 'recorded']
4. The largest recorded snowflake was in Keogh, MT during year 1887, and was 15 inches wide.
102. More than 50% of the people in the world have never made or received a telephone call.
134. A person can live without food for about a month, but only about a week without water. If the amount of water in your body is reduced by just 1%, you'll feel thirsty. If it's reduced by 10%, you'll die.


## 2.2. [15] Handling typos

### 2.2.0. Dataset 

Download github typo dataset from [here](https://github.com/mhagiwara/github-typo-corpus).
Load it with this code:

In [19]:
!pip install jsonlines

In [20]:
import jsonlines
from collections import Counter
import nltk
nltk.download('punkt')

dataset_file = "github-typo-corpus.v1.0.0.jsonl"

dataset = []
other_langs = set()

with jsonlines.open(dataset_file) as reader:
    for obj in reader:
        for edit in obj['edits']:
            if edit['src']['lang'] != 'eng':
                other_langs.add(edit['src']['lang'])
                continue

            if edit['is_typo']:
                src, tgt = edit['src']['text'], edit['tgt']['text']
                if src.lower() != tgt.lower():
                    dataset.append((edit['src']['text'], edit['tgt']['text']))
                
print(f"Dataset size = {len(dataset)}")

[nltk_data] Downloading package punkt to /home/ravio_o/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Dataset size = 245909


#### Explore sample typos
Please, explore the dataset. You may see, that this is
- mostly markdown
- some common mistakes with do/does
- some just refer to punctuation typos (which we do not consider)

In [21]:
for pair in dataset[1010:1020]:
    print(f"{pair[0]} => {pair[1]}")

        """Make am instance. =>         """Make an instance.
* travis: test agains Node.js 11 => * travis: test against Node.js 11
The parser receive a string and returns an array inside a user-provided  => The parser receives a string and returns an array inside a user-provided 
CSV data is send through the `write` function and the resulted data is obtained => CSV data is sent through the `write` function and the resulting data is obtained
One useful function part of the Stream API is `pipe` to interact between  => One useful function of the Stream API is `pipe` to interact between 
source to a `stream.Writable` object destination. This example available as  => source to a `stream.Writable` object destination. This example is available as 
`node samples/pipe.js` read the file, parse its content and transform it. => `node samples/pipe.js` and reads the file, parses its content and transforms it.
Most of the generator is imported from its parent project [CSV][csv] in a effort  => Most o

#### 2.2.0.1. Build a dataset vocabulary
Here we prepare a vocabulary for spellchecker testing and for estimating overall correction quality. Consider only word-level. Be carefull, there is markdown (e.g. \`name\`. \[url\]\(http://url)) and comment symbols (\#, //, \*).

In [22]:
def sent_to_words(sent):
    # splits sentence to words, filtering out non-alphabetical terms
    words = nltk.word_tokenize(sent)    
    words_filtered = filter(lambda x: x.isalpha(), words)
    return words_filtered

In [23]:
vocabulary = Counter()
for pair in dataset:
    for word in sent_to_words(pair[1].lower()):
        vocabulary[word] += 1
len(vocabulary)

63724

In [24]:
from itertools import islice
print(list(islice(vocabulary.items(), 10)))

[('function', 6193), ('de', 82), ('deutsch', 4), ('nocomments', 2), ('you', 42075), ('can', 26027), ('disable', 532), ('comments', 360), ('for', 44756), ('the', 207017)]


### 2.2.1. [15] Implement context-independent spellcheker (Trigrams with Jaccard coefficient) ##

In [25]:
def fix_typo_kgram(word, k_gram_index) -> list:
    #TODO return best matches with respect to Jaccard index


    def make_k_grams(candidate):
        # I use trigram splitting without $ sign, since it gives results closer to original word typing
        # With $ there are some permutation between same Jaccard coef terms
        k_gram_list = [candidate[i:i+k] for i in range(len(candidate)-k+1)]
        return k_gram_list

    def calc_jaccard(candidate):
        candidate_grams = make_k_grams(candidate)
        intersection = grams.intersection(candidate_grams)
        union = grams.union(candidate_grams)
        return len(intersection) / len(union)

    k = len(list(k_gram_index.keys())[0])
    grams = set(make_k_grams(word))
    candidates = set()
    for gram in grams:
        if gram in k_gram_index:
            set_g = set(k_gram_index[gram])
            candidates = candidates.union(set_g)
    # In general first 3-5 top candidated answer the need, others is just not null Jaccard coef terms that not make sence for original spelling/typing
    return sorted(candidates, key=calc_jaccard, reverse=True)[:5] or [word]

### 2.2.2. Tests

In [26]:
# tests

k_gram_index_github = build_k_gram_index(vocabulary, 3)
print(fix_typo_kgram("enouh", k_gram_index_github)[:20])
assert "enough" in fix_typo_kgram("enouh", k_gram_index_github), "Assert k-gram failed"

['enough', 'enought', 'eno', 'renounce', 'exogenous']


## 2.3. [Bonuses]

### 2.3.1. [Bonus] QWERTY - Editorial distance

Write code to compute weighted QWERTY-editorial distance.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/KB_United_States.svg/640px-KB_United_States.svg.png" width="640"/> 

Use this image to prepare weight function:
- all letter pairs which share vertical border will get 0.5 multiplier **for replace** (`df`, `cv`, `ui`, ...)
- all letter pairs which share at least some horizontal border will get 0.7 multiplier **for replace** (`dc`, `dr`, `km`, ...)
- other operations are not scaled (x1 multiplier).

In [27]:
import numpy as np
def replace_weight(let1, let2) -> float:
    # TODO what is the weight for a pair of letters being replaced? Note this function should be commutative

    board = np.array(["qwertyuiop","asdfghjkl$","zxcvbnm$$$"])
    row1 = [x for x in range(3) if let1 in board[x]][0]
    row2 = [x for x in range(3) if let2 in board[x]][0]
    col1 = [y for y in range(10) if let1 in board[row1][y]][0]
    col2 = [y for y in range(10) if let2 in board[row2][y]][0]

    if row1 == row2:
        if abs(col1-col2) == 1:
            return 0.5
    else:
        if row1 - row2 == 1:
            if col1==col2 or col1 + 1 == col2:
                return  0.7
        if row1 - row2 == -1:
            if col1==col2 or col2 + 1 == col1:
                return  0.7
    return 1

def qwerty_edit_dist(s1, s2) -> float:
    # TODO compute the Damerau-Levenshtein distance between two given strings (s1 and s2)
    # Parts of the code taken from: https://www.python-course.eu/levenshtein_distance.php
    rows = len(s1)+1
    cols = len(s2)+1
    dist = np.zeros((rows,cols))

    # source prefixes can be transformed into empty strings
    # by deletions:
    for i in range(1, rows):
        dist[i][0] = i

    # target prefixes can be created from an empty source string
    # by inserting the characters
    for i in range(1, cols):
        dist[0][i] = i

    for row in range(1, rows):
        for col in range(1, cols):
            if s1[row-1] == s2[col-1]:
                dist[row][col] = min(dist[row-1][col] + 1,      # deletion
                                     dist[row][col-1] + 1,      # insertion
                                     dist[row-1][col-1])        # substitution
            else:
                dist[row][col] = min(dist[row-1][col] + 1,      # deletion
                                     dist[row][col-1] + 1,      # insertion
                                     dist[row-1][col-1] + replace_weight(s1[row-1],s2[col-1]))    # substitution
            if row and col and s1[row-1]==s2[col-2] and s1[row-2] == s2[col-1]:
                    dist[row][col] = min (dist[row][col],
                                          dist[row-2,col-2] + 1) # transposition

    return dist[rows-1,cols-1]

In [28]:
# tests

assert qwerty_edit_dist("korrectud", "corrected") == 2.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("soem", "some") == 1.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("one", "one") == 0.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("fihure", "figure") == 0.5, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("fivure", "figure") == 0.7, "Edit distance is computed incorrectly"
## There is impossible to get 0.6 sin

### 2.3.2. [Bonus] Norvig's spellchecker with QWERTY weights

You can base your code on [Norvig's corrector](https://norvig.com/spell-correct.html), but you should be sure you account the fact, that typos for close letters cost less. This should be considered in ranking.

In [29]:
def fix_typo_qwerty_norvig(word) -> str:
    def P(word, N=sum(vocabulary.values())):
        "Probability of `word`."
        return vocabulary[word] / N

    def candidates(word):
        "Generate possible spelling corrections for word."
        return (known(edits1((word,0))) | known(edits2(word)))

    def known(words):
        "The subset of `words` that appear in the dictionary of WORDS."
        return set(w for w in words if w[0] in vocabulary)

    def edits1(word):
        "All edits that are one edit away from `word`."
        term = word[0]
        score = word[1]
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(term[:i], term[i:])    for i in range(len(term) + 1)]
        deletes    = [(L + R[1:],score + 1)               for L, R in splits if R]
        transposes = [(L + R[1] + R[0] + R[2:],score + 1) for L, R in splits if len(R)>1]
        replaces  = [(L + c +R[1:], score + replace_weight( R[0],c))          for L, R in splits if R for c in letters]
        inserts    = [(L + c + R,score + 1)               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(word):
        "All edits that are two edits away from `word`."

        res =  set()
        for e1 in edits1((word,0)):
            res.update(edits1(e1))
        return res

    res_sorted = sorted(list(candidates(word)), key = lambda x: x[1])
    if not known([(word,0)]) and len(res_sorted) == 0:
        return word
    min_koef = res_sorted[0][1]
    min_list = [res for res in res_sorted if res[1] == min_koef]
    if (known([(word,0)])):
        min_list.append((word,0))
    if len(min_list) > 1:
        return max(min_list, key=P)[0]
    else:
        return min_list[0][0]


In [30]:
# tests

assert fix_typo_qwerty_norvig("korrectud") == "corrected", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("speling") == "spelling", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("condidence") == "confidence", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("fpx") == "fox", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("fux") == "fix", "Norvig's correcter doesn't work"

### 2.3.3. Estimate quality of functions

In [31]:
norvig, kgram = 0, 0
limit = 10000
counter = limit
for i, (src, target) in enumerate(dataset):
    if i == limit:
        break
    words = sent_to_words(src.lower())
    # word suspected for typos
    sn, sk = src.lower(), src.lower()
    for word in words:
        if word not in vocabulary and word.isalpha():
            # top-1 accuracy
            wn, wk = fix_typo_qwerty_norvig(word), \
                     fix_typo_kgram(word, k_gram_index_github)[0]
            sn = sn.replace(word, wn)
            sk = sk.replace(word, wk)
    norvig += int(sn == target.lower())
    kgram += int(sk == target.lower())

print(f"Norvig accuracy ({norvig}) = {norvig / limit}")
print(f"k-gram accuracy ({kgram}) = {kgram / limit}")

IndexError: list index out of range